## Virtual distillation with classical shadows 

Related paper [Seif et al, PRX Quantum 4, 010303 2023](https://journals.aps.org/prxquantum/abstract/10.1103/PRXQuantum.4.010303)

Let a density matrix $\rho=(1-\epsilon)\ket{\psi}\bra{\psi}+\sum_\nu \epsilon_{\nu} \ket{\psi_{\nu}}\bra{\psi_{\nu}}$ represent an experimental state subject to noise, where $\ket{\psi}$ is the targeted pure state.

The idea of virtual distillation is that the rescaled state $\rho^n/\mathrm{tr}(\rho^n)$ converges to $\psi$ as $n$ increases. 

This means that the extraction of an observable $O$ is better done by evaluating $\mathrm{tr}(O\rho^n)/\mathrm{tr}(\rho^n)$ rather than using the standard estimator $\mathrm{tr}(O\rho)$. Such multi-copy observables can be estimated easily in the classical shadow framework.

In [2]:
using ITensors,ITensorMPS
using RandomMeas
using ProgressMeter

For this tutorial, we first load via DMRG the ground state of the transverse Ising model. 

In [3]:
N = 4
ξ = siteinds("Qubit", N)
B = 1.
ampo = AutoMPO()
for j in 1:(N - 1)
  # Ising ZZ interactions
  ampo .+= -1, "X", j, "X", j + 1
end
for j in 1:N
  # Transverse field X
  ampo .+= -B, "Z", j
end
H = MPO(ampo,ξ)
# Density-matrix renormalization group
dmrg_iter = 5      # DMRG steps
dmrg_cutoff = 1E-10   # Cutoff
ψ0 = random_mps(ξ) # Initial state
sweeps = Sweeps(dmrg_iter)
maxdim!(sweeps, 10, 20, 30, 40, 50, 100)
cutoff!(sweeps, dmrg_cutoff)
# Run 
println("Running DMRG to get ground state of transverse field Ising model:")
E, ψ = dmrg(H, ψ0, sweeps)
println("\nGround state energy:  ", E)
println("\n---------------------------------------\n")

Running DMRG to get ground state of transverse field Ising model:
After sweep 1 energy=-4.745313419333967  maxlinkdim=4 maxerr=4.87E-18 time=24.225
After sweep 2 energy=-4.758759954696696  maxlinkdim=4 maxerr=0.00E+00 time=0.003
After sweep 3 energy=-4.758770472473042  maxlinkdim=4 maxerr=2.11E-16 time=0.005
After sweep 4 energy=-4.75877048313254  maxlinkdim=4 maxerr=1.12E-16 time=0.004
After sweep 5 energy=-4.758770483143621  maxlinkdim=4 maxerr=0.00E+00 time=0.003

Ground state energy:  -4.758770483143621

---------------------------------------



Then we model the experimetal quantum state with global depolarization

In [5]:
#Consider experimental state admixed with white noise
p = 0.05
Ide = MPO(ξ)
for i in 1:N
    Ide[i] = δ(ξ[i]',ξ[i])
end
ρ = (1-p)*outer(ψ',ψ)+p*Ide/2^N;

And perform randomized measurements

In [6]:
nu = 500
NM = 100
data = zeros(Int8,(nu,NM,N));
u = Vector{Vector{ITensor}}()
ITensors.disable_warn_order()
@showprogress dt=1 for r in 1:nu
        push!(u,get_rotations(ξ,1))
        data[r,:,:] =  get_RandomMeas(ρ,u[r],NM)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:30


We then used three batch classical shadows [Rath et al, PRX quantum 2023](https://doi.org/10.1103/PRXQuantum.4.010318) for postprocessing the data.

In [9]:
n = 3 #number of batch shadows
ρb = get_batch_shadows(data,ξ,u,n);

The three batch shadows can be used to form unbiased estimates of $\rho,\rho^2,\rho^3$ respectively, using U-statistics [Huang et al, Nat Phys 2020](https://doi.org/10.1038/s41567-020-0932-7)

In [10]:
ρm = Vector{ITensor}(undef,n)
ρm[1] = (ρb[1]+ρb[2]+ρb[3])/3/nu
ρm[2] = (multiply(ρb[1],ρb[2])+multiply(ρb[1],ρb[3])+multiply(ρb[2],ρb[3]))/3/nu
ρm[3] = multiply(multiply(ρb[1],ρb[2]),ρb[3])/nu;

We can then evaluate $\mathrm{tr}(O\rho^n)/\mathrm{tr}(\rho^n)$ for $n=1,2,3$.

In [11]:
Es = zeros(n)
for nt in 1:n
    Es[nt] = real(trace(multiply(ρm[nt],flatten(H)),ξ)/trace(ρm[nt],ξ))
end
println("Estimations for different purification order n " , Es)
println("Real values ",E)

Estimations for different purification order n [-4.482176775009767, -4.784397490676381, -4.7554420427271396]
Real values -4.758770483143621
